# The math behind landing burns

### And not just ones in KSP!

A lot of different elements go into timing your landing burn right. With some math, we can make a kOS script for vertical landing that works even better than KER's burn prediction!

We need to consider the following, sorted from easiest to implement to hardest:

1. Thrust vs weight and gravity
2. Local gravity
3. Change in thrust
4. Aerodynamic Drag
5. Change in air density
6. Change in mass

## Definitions

 - Velocity $V$ is speed and direction in $m/s$
 - Acceleration $A$ is change in velocity in $m/s^2$
 - Altitude $H$ is height above sea level in $m$
 - Radar Altitude $R$ is height above ground in $m$
 - Time $t$ is a time in seconds $s$
 - Mass $M$ is mass in metric $tons$ or $1000 kg$
 - Thrust $F$ is total engine thrust in kilonewtons $kn$
 - Offset $O$ is the radar offset in $m$ (The radar altitude readout when landed)

We will add more variables later.

A variable with no subscript is treated as a constant, such as $g$\
A variable with subscript 0 is instantaneous, such as $V_0$ being the current velocity\
A variable with subscript a is a predicted average, such as $A_a$ being the average acceleration\
A variable with subscript t is a target, such as $R_t=0$ being the target landing altitude

### Some essential equations

<center>
$$F=MA$$ Newton's second law
</center>
<br>
<center>
$$F=\Delta M*V_e$$ The Rocket Equation, where the upwards force is exhaust mass times exhaust velocity
</center>
<br>
    
<center>
$$t=-V_0/A$$ Time needed to cancel current velocity at constant acceleration
</center>
<br>
    
<center>
$$V_a=(V_0+V_t)/2$$ Average of velocity, assuming constant acceleration
</center>

## Part 1: Thrust, Mass, and Gravity

We can find acceleration of the rocket engines by reorganizing $F=MA$:$$A=\dfrac{F}{M}$$

For now, we assume that gravitational acceleration is $g=-9.81m/s$, so we can add that to our equation:
$$A=\dfrac{F}{M}-9.81$$

Then we want to find the burn distance, which will be the height we start our burn. We know our average speed $V_a$ is $\dfrac{V_0}{2}m/s$ and time $\dfrac{-v_0}{A}s$, so we can multiply them and take the absolute value. To find the true altitude we need to burn at, we need to also add the height offset $O$ of the ship:
$$R=\mid\dfrac{-V_0^2}{2A}\mid + O$$

Tada! If you went to a planet where $g=9.81$, there is no atmosphere, and you turn on Infinite Fuel in KSP to keep your ship mass constant, this equation would give you a picture-perfect vertical landing. Unfortunately, none of these hold.

## Part 2: Local Gravity

If you are like me, you have probably been told that acceleration due to gravity is $g=9.81m/s^2$. It is actually dependent on your distance from the body. You could probably get away with using 9.81, but it is not hard to implement a smarter solution.

Gravitational acceleration on Earth (or Kerbin) follows this equation:
$$g=\dfrac{G*M_e}{r^2}$$
or the gravitational parameter $G$, planet or body mass $M_e$, and distance from center $r$.

The product $G*M_e$ can also be written as $\mu$, where $\mu$ is constant for a body. Our distance from the center is the planetary radius plus our height above sea level. Therefore, we get:
$$g=\dfrac{\mu}{(r_e+H)^2}$$

This would be fine if we held the same altitude, but we want an average. We can predict the average acceleration when falling a certain distance with the following:
$$g_a=-\dfrac{\mu}{r_e(r_e+H)}$$

Instead of substituting this equation into our acceleration equation,we will just use g:
$$A=\dfrac{F}{M}+g_a$$

Keep in mind that we are adding $g_a$ because it is inherently negative (it accelerates us down).

## Part 3: Change in thrust

Right now we are assuming that our thrust, $F$, is constant, but in reality it changes as a function of your altitude. To simplify the math, we will make a few assumptions:

1. Acceleration is constant. We will compute the average acceleration
2. Velocity, being the integration of acceleration, is linear
3. Altitude, being the integration of velocity, is quadratic

We will say $F_0$ is thrust at currrent altitude, and $F_p$ is the sea_level thrust.

Since $F$ is a function of altitude, it is quadratic in relation to time. To find the average of a function $f(x)=ax^2+c$ from r to s, we can do: 
$$\dfrac{a(s^3-r^3)}{3(s-r)}+c=\dfrac{a(s^2+sr+r^2)}{3}+c$$

This scales up easily:
$$f(x)=ax^n+c$$
$$f_a=\dfrac{a(s^{n+1}-r^{n+1})}{(n+1)(s-r)}+c$$

C will be the value at our landing target. Therefore, $ar^2$ will be 0, and $as^2+c$ will be the current value of whatever variable we are averaging.

Therefore:
$$F_a=\dfrac{F_0-F_p}{3}+F_p$$
$$F_a=\dfrac{F_0+2F_p}{3}$$

Our new acceleration equation is:
$$A=\dfrac{F_a}{M}+g_a$$

This math is getting fairly complicated so far, but we still have a clear set of equations. Aerodynamic drag is a bit easier to calculate, but it is not very theoretical in comparison to our past work.

## Part 4: Aerodynamic drag

To find drag, we get a three new variables: $d$ is the acceleration due to drag, $c_d$ is the coefficient of drag, and $p$ is the air pressure relative to sea level. On Earth, this would either be 1 bar or atm. Acceleration due to drag can be expressed with the following equation:
$$d_0=pc_dV_0^2$$

Since we are using $V_0^2$, we need to do a quadratic average. Fortunately, it is a lot easier to calculate this, because our final velocity will be 0.
$$d_a=pc_d\dfrac{V_0^2}{3}$$

To find the coefficient of drag, we must do some experimentation. In KSP, I dropped my ship from 10 km and recorded pressure, velocity, and acceleration from 1km to impact. If you want to do this in real life, just use a wind tunnel. You must take air pressure changes and ship orientation into account, or your numbers will not be accurate. My result was $c_d=0.000152$; if you are using my booster the number will be the same.

This brings the first addition to our acceleration equation, and our first consideration of the atmosphere! We are adding drag: we want to slow down, so it is contributing to our accelerating power.
$$A=\dfrac{F_a}{M}+g_a+d_a$$

## Part 5: Change in air pressure

To find average air pressure, we need to make another assumption: air pressure is roughy linear. Given this assumption, it would be proportional to altitude. We already made the assumption that altitude is quadratic with respect to time, so we can bring this forward and say air pressure is quadratic with respect to time.

We must bring back our quadratic term average equation:
$$f(x)=ax^2$$
$$\dfrac{a(s^2+sr+r^2)}{3}$$

If you want extreme precision or are landing at high altitudes, you should use an equation to find pressure given an altitude or find it experimentally. I'm landing at between 74-0 meters above sea level, so I am going to assume final pressure is 1 atm. You will need a vessel sensor or pressure equation to find initial pressure.

Given $p_0$ and $p_p$, we can find the average with:
$$p_a=\dfrac{p_0 + 2p_p}{3}$$

If you are landing at 1 atm, you get this simpler equation:
$$p_a=\dfrac{p_0+2}{3}$$

We can put this into our drag equation:
$$d_a=p_ac_d\dfrac{V_0^2}{3}$$

And our acceleration equation remains the same:
$$A=\dfrac{F_a}{M}+g_a+d_a$$

## Part 6: Change in mass
### The Hard Part

Finding how the mass changes is trickier than anything we have done yet. Why?

To find the change in mass and thus its average, we need to know the amount and total mass of burnt fuel. To find this, we need the burn time. To find the burn time, we need the acceleration. Finally, to find acceleration, we need the average mass.

We'll start by finding the weight of burnt fuel. We need to know fuel burn rate $r$ in $l/s$ and fuel weight $M_f$ in $tons/l$. The burn rate is linear with time. Given these, plus time, we can find an equation to find the change in mass:
$$\Delta M=trM_f$$
Then the average:
$$M_a=\dfrac{M_0 + (M_0-\Delta M)}{2}=M_0-\dfrac{\Delta M}{2}$$

We can also recall that $t=\dfrac{-V_0}{A}$

Substituting the above equations, we can find:
$$M_a=M_0+\dfrac{-V_0rM_f}{a}$$

We can define two variables to simplify our system of equations:

<center>The part of acceleration independent of mass (constant of acceleration)</center>
$$c_a-g$$

<center>The coefficient of the inverse of acceleration in the fuel change equation (fuel coefficient)</center>
$$f=-V_0rM_f$$

Using these variables, we can simplify our system significantly:
$$A=\dfrac{F_a}{M_a}+c_a$$
$$M_a=M_0+\dfrac{f}{A}$$

Now, there are two approaches we can take. First is the iterative approach, where we choose a startign value for one of the variables, solve the other equation, and keep plugging values in until you are satisfied with the covergence. This does not require annoyingly long algebra, but needs more computation power and is less accurate. I did this initially, but found it was either too slow (many iterations) or too inaccurate (fewer iterations).

The second is to solve the system of equations, which is both fast and accurate. I have done it (and messed up a few times) and you get some very unfriendly numbers - but as soon as you plug this into a program, it can very quickly return the exact numbers you need. This is the route we will choose here.

I highly recommend that you solve the system for either variable yourself instead of blindly plugging in this equation - it will help you learn what is going on and it will be more gratifying.

I solved for acceleration, as that is what you need to find your burn height. You don't really even need to solve for average mass after that, but I do for debug info. This is the quadratic for acceleration:

$$(M_0)x^2 + (f+F_a+M_0c_a)x + (c_af) = 0$$

We can solve it using the quadratic formula:
$$ax^2+bx+c=0$$
$$r,s=\dfrac{-b\pm\sqrt{b^2-4ac}}{2a}$$
(expansion not pictured)

Finally, we get two roots. The one we want will always be the greater (positive) one, and the other will be an extraneous solution. We do not need to compare them; the greater root will always be:
$$x=\dfrac{-b+\sqrt{b^2-4ac}}{2a}$$

So then we get that our final acceleration is described by the following:
$$a=M_0$$
$$b=f=F_a+M_0c_a$$
$$c=c_af$$
$$A=\dfrac{-b+\sqrt{b^2-4ac}}{2a}$$

And for the final step, we bring back our burn height equation:
$$R=\mid\dfrac{-V_0^2}{2A}\mid + O$$
And we are left with the height we need to burn at the current state, given our many variables. The only thing left to do is iterate over our formula until our current height is at our burn height.

## Final notes: Calculus, Accuracy, and the Unpredictable

This project has the goal of not touching calculus, given that I don't know it yet (except for chain rule). I did give you the equations for averaging higher-power functions, which requires integration to do on your own, but they are simply expressed and make sense, so I am discounting them. 

During these calculations, we made 4 assumptions about the math:

1. Acceleration is constant. It is not, but we solved this by finding the average.
2. Velocity is linear. It is again not, because acceleration is not. This may affect some of our calculations.
3. Height is quadratic. You know the drill by now.
4. Time is linear. No, I'm not talking about relativity, a few nanoseconds (if that) won't break your burn. I'm talking about compute time. Computers do not run programs sequentially, instead scheduling them to run based on priority. In KSP, physics ticks, which are when calculations occur, can be variable in length. You must take this into account, especially with kOS. To counteract this with kOS, you can bump up speed with `set config:ipu to 2000`. The default "instructions per update" is 200, and it can go down to 5. In real life, your data may vary.

The first three can be fixed with more calculations, but those exagerate the problems of the fourth point. Generally, this approach should give you a very accurate and precise number, down to a few meters if that. If you need even more precision and use throttle control, you can use a P or PI loop to fine tune your thrust based on current height vs burn height. You could also use airbrakes to adjust your drag.

Another question you may have is "If you changed the mass iteration to a run-once formula, why not do it for the height checking? Why iterate over it?"

The short answer is accuracy. It is significantly harder to predict your entire descent compared to just the part where you burn.

The problem is the unknowns: horizontal velocity complicates the process significantly, if you are using fins or thrusters to steer, it will interfere with your calculations, you must take into account the full atmosphere, which does not have one complete equation but instead four different ones based on altitude, and a number of other factors that you may find. Even a simulator cannot predict everything. Can it be done? Maybe, but is it worth it?

An idea that may be worth pursuing is smarter iterations. From one altitude, calculate the burn height with a less complicated complete equation, and when you get close, redo it until your resut matches your altitude. It is certainly complicated, but much more doable.

There remain a few things you cannot predict in real life: weather and its effects, bit flips if you are actually going to space (which I hope you aren't if you are doing the math based on a 15 year old's tutorial), and animals such a birds, just to name a few. Weather is by far the factor with the most influence: a gust of wind may push you off of your target, waves may rock your drone ship, thunder may strike your vehicle. There may also be some variables that you do not know.

Fixing this is another rabbit hole: one of PID loops, feedback loops, and nonlinear control to correct for error, Kalman filters and advanced approximation for unknowns, factoring elements like windspeed at various altitudes in your descent into your calculations.

If you want a controlled environment to test things, I would recommend Kerbal Space Program (KSP). If you are reading this, you most likely already know of KSP, because that is what this repo is designed for. If you have all of your math down for KSP, however, try weather or error mods, or if you have the time and means, real life.